In [94]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import pickle 
import os 
import sys
import warnings

from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier 

sys.path.append(os.path.abspath(".."))
from cleaning_preprocess_ML.cleaning import datacleaning
from cleaning_preprocess_ML.cleaning import preprocess_data


In [95]:
path = os.getcwd()
chemin_repertoire = os.path.join(path, '../dataset')

# chemin des fichiers
chemin= os.path.join(chemin_repertoire, 'data_cleaned.csv')

In [96]:
df = pd.read_csv(chemin)
df.head(5)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
0,5849,0.0,128.0,360.0,1.0,Male,No,0,Graduate,No,Urban,Y
1,4583,1508.0,128.0,360.0,1.0,Male,Yes,1,Graduate,No,Rural,N
2,3000,0.0,66.0,360.0,1.0,Male,Yes,0,Graduate,Yes,Urban,Y
3,2583,2358.0,120.0,360.0,1.0,Male,Yes,0,Not Graduate,No,Urban,Y
4,6000,0.0,141.0,360.0,1.0,Male,No,0,Graduate,No,Urban,Y


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ApplicantIncome    614 non-null    int64  
 1   CoapplicantIncome  614 non-null    float64
 2   LoanAmount         614 non-null    float64
 3   Loan_Amount_Term   614 non-null    float64
 4   Credit_History     614 non-null    float64
 5   Gender             614 non-null    object 
 6   Married            614 non-null    object 
 7   Dependents         614 non-null    object 
 8   Education          614 non-null    object 
 9   Self_Employed      614 non-null    object 
 10  Property_Area      614 non-null    object 
 11  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 57.7+ KB


#### Feature Engineering

In [98]:
type(df)

pandas.core.frame.DataFrame

In [99]:
df

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
0,5849,0.0,128.0,360.0,1.0,Male,No,0,Graduate,No,Urban,Y
1,4583,1508.0,128.0,360.0,1.0,Male,Yes,1,Graduate,No,Rural,N
2,3000,0.0,66.0,360.0,1.0,Male,Yes,0,Graduate,Yes,Urban,Y
3,2583,2358.0,120.0,360.0,1.0,Male,Yes,0,Not Graduate,No,Urban,Y
4,6000,0.0,141.0,360.0,1.0,Male,No,0,Graduate,No,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
609,2900,0.0,71.0,360.0,1.0,Female,No,0,Graduate,No,Rural,Y
610,4106,0.0,40.0,180.0,1.0,Male,Yes,3+,Graduate,No,Rural,Y
611,8072,240.0,253.0,360.0,1.0,Male,Yes,1,Graduate,No,Urban,Y
612,7583,0.0,187.0,360.0,1.0,Male,Yes,2,Graduate,No,Urban,Y


In [100]:
y = df['Loan_Status']

In [ ]:
X = df.drop('Loan_Status', axis=1, inplace=True)

In [105]:
y

0      Y
1      N
2      Y
3      Y
4      Y
      ..
609    Y
610    Y
611    Y
612    Y
613    N
Name: Loan_Status, Length: 614, dtype: object

In [79]:
X = preprocess_data(X)

y_value= {'Y':1, 'N':0}
y = y.map(y_value)

In [ ]:
# division des données en données d'entrainement et données de test
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train, test in sss.split(X,y) :
    X_train, X_test = X.iloc[train], X.iloc[test]
    y_train, y_test = df[y].iloc[train], df[y].iloc[test]
    
print('X_train taille :', X_train.shape)
print('X_test taille :', X_test.shape)
print('y_train taille :', y_train.shape)    
print('y_test taille :', y_test.shape) 

In [84]:
y.shape

(614,)

In [85]:
X.shape

AttributeError: 'tuple' object has no attribute 'shape'